In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from fredapi import Fred
import random

random.seed(42)

#Load FRED data
f = open("fred_key.txt")
fred = Fred(api_key=f.read())

# Inflation (CPI YoY %), Output gap (CBO estimate), Unemployment rate
cpi = fred.get_series('CPIAUCSL').pct_change(12).dropna() * 100         # annual inflation %
gdp = fred.get_series('GDPC1')                                          # real GDP
potential_gdp = fred.get_series('GDPPOT')
output_gap = 100 * (gdp - potential_gdp) / potential_gdp

unemployment = fred.get_series('UNRATE')

# Align into one DataFrame
df = pd.concat([cpi, output_gap, unemployment], axis=1)
df.columns = ['inflation', 'output_gap', 'unemployment']
df = df.dropna()

In [2]:
# Estimate Phillips Curve gamma

df['inflation_lead'] = df['inflation'].shift(-1)
df = df.dropna()

X = sm.add_constant(df[['inflation_lead', 'output_gap']])
y = df['inflation']
phillips_model = sm.OLS(y, X).fit()
print(phillips_model.summary())

                            OLS Regression Results                            
Dep. Variable:              inflation   R-squared:                       0.901
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     1379.
Date:                Tue, 19 Aug 2025   Prob (F-statistic):          1.40e-152
Time:                        21:55:55   Log-Likelihood:                -399.97
No. Observations:                 305   AIC:                             805.9
Df Residuals:                     302   BIC:                             817.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.1433      0.082      1.

In [3]:
gamma = phillips_model.params['output_gap']
beta_inflation = phillips_model.params['inflation_lead']
print("gamma = ", np.round(gamma,4))
print("expected inflation = ", np.round(beta_inflation,4))

gamma =  -0.0822
expected inflation =  0.9534


In [4]:
# Estimate IS Curve

rate = fred.get_series('FEDFUNDS')
df['rate'] = rate
df['output_gap_lead'] = df['output_gap'].shift(-1)
df['inflation_lead'] = df['inflation'].shift(-1)
df = df.dropna()

X_is = sm.add_constant(df[['output_gap_lead', 'rate', 'inflation_lead']])
y_is = df['output_gap']
is_model = sm.OLS(y_is, X_is).fit()
print(is_model.summary())

                            OLS Regression Results                            
Dep. Variable:             output_gap   R-squared:                       0.805
Model:                            OLS   Adj. R-squared:                  0.802
Method:                 Least Squares   F-statistic:                     381.4
Date:                Tue, 19 Aug 2025   Prob (F-statistic):           3.45e-98
Time:                        21:55:55   Log-Likelihood:                -395.53
No. Observations:                 282   AIC:                             799.1
Df Residuals:                     278   BIC:                             813.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.3321      0.101     

In [5]:
rate = is_model.params['rate']
output_gap_lead = is_model.params['output_gap_lead']
inflation_lead = is_model.params['inflation_lead']
print("Interest Rate: ", round(rate,4))
print("Expected Output Gap: ", round(output_gap_lead,4))
print("Expected Inflation: ", round(inflation_lead,4))

Interest Rate:  0.0463
Expected Output Gap:  0.8914
Expected Inflation:  0.0216


In [6]:
# Estimate Shock Process
residuals = phillips_model.resid
df['shock'] = residuals
df['shock_lag'] = df['shock'].shift(1)
df = df.dropna()

shock_model = sm.OLS(df['shock'], sm.add_constant(df['shock_lag'])).fit()
print(shock_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  shock   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     6.285
Date:                Tue, 19 Aug 2025   Prob (F-statistic):             0.0127
Time:                        21:55:55   Log-Likelihood:                -332.10
No. Observations:                 281   AIC:                             668.2
Df Residuals:                     279   BIC:                             675.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0120      0.047     -0.254      0.8

In [7]:
rho_u = shock_model.params['shock_lag']
shock_std = shock_model.resid.std()
print("Shock Standard Deviation: ",round(shock_std,4))

Shock Standard Deviation:  0.7903
